In [1]:
import torch
from finetune import Resnet18Finetuner, Resnet18FintunerForCaseD
from dataloader import load_augmented_data

In [2]:
baseline = Resnet18Finetuner(pretrained=True, feature_extract=True)
model_A = Resnet18Finetuner(pretrained=True, feature_extract=True)
model_B = Resnet18Finetuner(pretrained=True, feature_extract=False)
model_C = Resnet18FintunerForCaseD(pretrained=True, feature_extract=True)

/opt/homebrew/Caskroom/miniconda/base/envs/dac/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/dac/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
baseline.model.layer1

Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (1): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

In [7]:
data_dir = "../face_dataset"

## Set hyperparameters and share with all cases

In [8]:
input_size = 64
batch_size = 256
num_epochs = 20

In [9]:
dataloaders_dict = load_augmented_data(input_size, batch_size, data_dir)
dataloaders_dict

Initializing Datasets and Dataloaders...


{'train': <torch.utils.data.dataloader.DataLoader at 0x127c20a90>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x127c21780>}

In [10]:
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# Case A: Re-train the softmax layer (Baseline model)

When finetuning all resnet layers, feature_extract = False, the model is finetuned and all model parameters are updated.  
If feature_extract = True, only the last layer parameters are updated, the others remain fixed.

In [11]:
baseline.model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [12]:
# Train and evaluate
baseline.model, hist_base = baseline.fit(dataloaders_dict, num_epochs=num_epochs)

Params to learn:
	 fc.weight
	 fc.bias
	 batch_norm.weight
	 batch_norm.bias
Epoch 0/19
----------


[E thread_pool.cpp:113] Exception in thread pool task: mutex lock failed: Invalid argument
[E thread_pool.cpp:113] Exception in thread pool task: mutex lock failed: Invalid argument
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x12003ec20>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/dac/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1471, in __del__
    self._shutdown_workers()
  File "/opt/homebrew/Caskroom/miniconda/base/envs/dac/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1435, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/opt/homebrew/Caskroom/miniconda/base/envs/dac/lib/python3.10/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/opt/homebrew/Caskroom/miniconda/base/envs/dac/lib/python3.10/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
  File "/opt

KeyboardInterrupt: 

In [ ]:
plt.plot(hist_base)

## Case B: Fine tune Conv5_x and freeze the rest of Conv blocks

In [ ]:
# Freeze all Conv blocks in Resnet18
model_A.unfreeze_layer("layer4")

When finetuning all resnet layers, feature_extract = False, the model is finetuned and all model parameters are updated.  
If feature_extract = True, only the last layer parameters are updated, the others remain fixed.

In [ ]:
# Train and evaluate
model_A.model, hist_A = model_A.fit(dataloaders_dict, num_epochs=num_epochs)

## Case C: Fine tune ALL convolution layers.

In [ ]:
# Train and evaluate
model_B.model, hist_B = model_B.fit(dataloaders_dict, num_epochs=num_epochs)

## Case D: Freeze all the convolution blocksk, introudce two FC layers prior to the softmax layer

In [ ]:
model_C.model, hist_C = model_C.fit(dataloaders_dict, num_epochs=num_epochs)

In [ ]:
plt.plot(hist_base)
plt.plot(hist_A)
plt.plot(hist_B)
plt.plot(hist_C)